# Warzone Lobby Stats

In [ ]:
import requests
import urllib.parse
import random
import os
import pandas as pd
import time
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Authentication: See the other two notebooks and videos on authentication.

In [ ]:
s = requests.Session()
s.get('https://profile.callofduty.com/cod/login')
data = {'username': os.environ.get('COD_EMAIL'), 
        'password': os.environ.get('COD_PW'), 
        'remember_me': 'true', 
        '_csrf': s.cookies['XSRF-TOKEN']}
s.post('https://profile.callofduty.com/do_login?new_SiteId=cod', params=data)

In [ ]:
match_id = "3726192899756849207"
team_mates = {"Vio": "14688938695022220470", "Bublous": "1860799185863297602"}
#, "Uncarriable": '5291500827578550226'
#team_mates = {"Huskerrs": "11946786255256953231", "Symfuhny": "4864369508081987227", "ZLaner": "5499179043424889056"}
#team_mates = {"BookaShade": "16503230311151154188"}
#team_mates = {"Huskerrs": "11946786255256953231", "ZLaner": "5499179043424889056", "OakBoi": "1506652682349281517", "AydaN": "6466570850733519449"}
#team_mates = {"ZLaner": "5499179043424889056", "Destroy": "13121215955392487462", "Trash_Fue": "9516497865377061135", "SuperEvan": "3099898901879856705"}

In january after some outcry about sbmmwarzone.com Activision decided to put all COD Profiles privacy on private. What this means is that getting a users profile data using their "uno" is not possible anymore. This was an easy way to calculate the average lobby kd because in the match endpoint all the players uno's were returned. Now even after a user sets it's profile to public you won't be able to get info about this user, only when he/she is your friend it's still possible.

Get data from a match/lobby:

In [ ]:
lobby_data = s.get('https://my.callofduty.com/api/papi-client/crm/cod/v2/title/mw/platform/battle/fullMatch/wz/' + match_id + '/it')

In [ ]:
lobby_data.json()['data']

In [ ]:
lobby_data.json()['data']['allPlayers'][0]

In [ ]:
#username = urllib.parse.quote("Weixo En Nike")
username = urllib.parse.quote("Viioozz")
username

In [ ]:
search_resp = s.get(f'https://my.callofduty.com/api/papi-client/crm/cod/v2/platform/uno/username/{ username }/search')
search_resp.json()

In [ ]:
lobby_players = lobby_data.json()['data']['allPlayers']
lobby_players_df = pd.DataFrame.from_records(lobby_players)
lobby_players_df.info()

In [ ]:
lobby_players_df['playerStats']

In [ ]:
lobby_players_df = pd.DataFrame.from_records(lobby_players_df['playerStats'])
lobby_players_df

In [ ]:
lobby_players_df.info()

In [ ]:
lobby_players_df.describe()

In [ ]:
resp = s.get('https://www.callofduty.com/api/papi-client/stats/cod/v1/title/mw/platform/uno/uno/' + str(lobby_players[43]['player']['uno']) + '/profile/type/warzone')

In [ ]:
resp.json()

In [ ]:
kds = []
lobby_players_full = []
team = ""
teamPlacement = ""
team_stats = {}
for player in lobby_players:
    time.sleep(0.2)
    if player['player']['uno'] in team_mates.values():
        team = player['player']['team']
        team_placement = player['playerStats']['teamPlacement']
        team_stats[player['player']['username']] = [player['playerStats']['kills']]
    resp = requests.get('https://www.callofduty.com/api/papi-client/stats/cod/v1/title/mw/platform/uno/uno/' + str(player['player']['uno']) + '/profile/type/warzone', headers=headers, cookies=resp_login.cookies)
#    try resp.json()['data'].get('message', ) == "Not permitted: not allowed":
#        kds.append('NA')
#    catch:
    kds.append(resp.json().get('data', {}).get('lifetime', {}).get('mode', {}).get('br', {}).get('properties', {}).get('kdRatio'))
    lobby_players_full.append(resp.json()['data'])
kds

In [ ]:
len(kds)

In [ ]:
plt.hist(kds, density=True, bins=50)
plt.xlabel("KD")
plt.ylabel("#Players")

In [ ]:
avg = round(sum(kds)/len(kds), 2)
math.floor(len(kds)/2)
median = round(sorted(kds)[math.floor(len(kds)/2)], 2)
kd_min = round(min(kds), 2)
kd_max = round(max(kds), 2)
top_ten = [round(kd, 2) for kd in sorted(kds)[math.floor(len(kds) * (80/100)):len(kds)]]
print("Game ID: " + match_id)
print("Team Placement: " + str(team_placement))
print("Team Kills")
print(team_stats)
print("Lobby Stats")
print("Average KD: " + str(avg))
print("Median KD: " + str(median))
print("Worst KD: " + str(kd_min))
print("Best KD:" + str(kd_max))
print("Top 20% KD: " + str([x for x in top_ten]))


In [ ]:
sorted_kds = sorted(kds)

In [ ]:
sorted_kds = sorted_kds[:-4]
adj_avg = round(sum(sorted_kds)/len(sorted_kds), 2)
print(sorted_kds[-1])
print(len(sorted_kds))
print(adj_avg)

In [ ]:
max(kds)

In [ ]:
min(kds)

In [ ]:
np.var(kds)

In [ ]:
resp_profile = requests.get('https://www.callofduty.com/api/papi-client/stats/cod/v1/title/mw/platform/uno/uno/12035878875057403705/profile/type/warzone', headers=headers, cookies=resp_login.cookies)
resp_profile.json()

In [ ]:
resp_profile_pub = requests.get('https://www.callofduty.com/api/papi-client/stats/cod/v1/title/mw/platform/uno/uno/14688938695022220470/profile/type/warzone', headers=headers, cookies=resp_login.cookies)
resp_profile_pub.json()

In [ ]:
resp_profile_pub.json()['data']['lifetime']['mode']['br']['properties']['kdRatio']

In [ ]:
'https://my.callofduty.com/api/papi-client/crm/cod/v2/title/mw/platform/uno/uno/4066176156082092197/matches/wz/start/0/end/0/details' \
--header 'Cookie: ACT_SSO_COOKIE=Set by test scripts; ACT_SSO_COOKIE_EXPIRY=1591153892430; atkn=Set by test scripts;'

In [ ]:
resp_profile = requests.get('https://www.callofduty.com/api/papi-client/stats/cod/v1/title/mw/platform/psn/gamer/Viioozz/profile/type/warzone', headers=headers, cookies=resp_login.cookies)
resp_profile.json()